In [102]:
#start/ importing begging libraries
import requests
import json
import csv  
import urllib.request
import pandas as pd
print("yn = yes or no")
print("green fn")
print("yellow fn")
print("red fn")
print("blue fn")

In [ ]:
#california natural resource agency 

url = 'https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=8ea91e09-3c84-4eba-9489-3ab3b08adfd0&limit=10000' 
# Send a GET request to the API
response = requests.get(url)
# Parse the JSON response
response_dict = response.json()
filtered_records = []
# Accessing records and printing the value of YEAR_ for each record
if "result" in response_dict and "records" in response_dict["result"]:
    for record in  response_dict["result"]["records"]:
        year_value = record.get("YEAR_")  # Access YEAR_ from each record
        try:
            # Only print the entire record where YEAR_ is 2000 or later
            if int(year_value) >= 2000:
               filtered_records.append(record)
        except (ValueError, TypeError):
            # Skip :ecords where YEAR_ is not a valid number
            continue

df = pd.DataFrame(filtered_records)
df.to_csv('cnra_data.csv', index=False, encoding='utf-8')
# Write the DataFrame to a CSV file
# filter the data by alarm_date

df1 = pd.read_csv('dont_delete/cnra_data.csv') # read the csv file
df1['ALARM_DATE'] = pd.to_datetime(df1['ALARM_DATE'], errors='coerce') # convert the alarm_date column to datetime
#df1['ALARM_DATE'] = df1['ALARM_DATE'].dt.tz_convert('UTC')
df1_sorted = df1.sort_values(by = 'ALARM_DATE', ascending=False) # filter the data by alarm_date
df1_sorted.to_csv('exported_data/cnra_data_sorted.csv', index=False) # save the filtered data to a new csv file

In [ ]:
#order by incident_date_created  
# Read the CSV file
df2 = pd.read_csv('dont_delete/fire_ca.csv')
fire = df2[df2['incident_name'].str.contains('Fire', case=False, na=False)].copy()
# copying onlie fire data
# Convert 'incident_date_created' to datetime, coercing errors to NaT
fire['incident_date_created'] = pd.to_datetime(fire['incident_date_created'], errors='coerce')
# Sort by 'incident_date_created' in descending order
df_sorted = fire.sort_values(by='incident_date_created', ascending=False)
# Save the sorted DataFrame to a new CSV file
df_sorted.to_csv('sorted_by_year.csv', index=False) 

In [104]:
df = pd.read_csv('exported_data/cnra_data_sorted.csv')
        
    # Remove multiple columns
columns_to_remove = ['OBJECTID', '_id', 'YEAR_','STATE','AGENCY','UNIT_ID','INC_NUM','OBJECTIVE','COMMENTS','IRWINID','FIRE_NUM','DECADES','COMPLEX_ID']
# OBJECTID
#_id
# YEAR_
# STATE
# AGENCY 
# UNIT_ID

# INC_NUM
# OBJECTIVE 
# COMMENTS 
# IRWINID 
# FIRE_NUM
# COMPLEX_ID 
# DECADES

df.drop(columns=columns_to_remove, axis=1, inplace=True)

In [ ]:
'this gets the data from google earth engine from landfire,'
'we will make it so that we collect the vegetation type(returns a value corresponding to the vegation)'
'the evc is the vegation cover, this returns a code corresponding to vegetation look at the landfire documentation for more information'

import ee
import pandas as pd

# Initialize EE
ee.Initialize(project='fuzz-464001')
# Define a point (Northern California coastal area)
point = ee.Geometry.Point([-120.87, 41.44])
# Load LANDFIRE datasets (corrected case sensitivity)
evc_image = ee.Image('LANDFIRE/Vegetation/EVC/v1_4_0/CONUS')
evt_image = ee.Image('LANDFIRE/Vegetation/EVT/v1_4_0/CONUS')  # <- Corrected
# Print band names
#print(evc_image)
print("EVC Bands:", evc_image.bandNames().getInfo())  # Should show ['EVC']
print("EVT Bands:", evt_image.bandNames().getInfo())  # Should show ['EVT']
# Get pixel values
evc_result = evc_image.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=point,
    scale=30  # 30m resolution
).getInfo()  # Directly get the Python dictionary

evt_result = evt_image.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=point,
    scale=30
).getInfo()

print("\nEVC Value (Cover %):", evc_result['EVC'])
print("EVT Value (Type Code):", evt_result['EVT'])


image_biomass = ee.Image('LARSE/GEDI/GEDI04_B_002')
layer1 = image_biomass.select('MU')
biomass_result = layer1.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=point,
    scale=30
).getInfo()

confidence_layer = image_biomass.select('PE')
confidence_result = confidence_layer.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=point,
    scale=30
).getInfo()




' for this we are going to use the GEDI dataset, which provides biomass and confidence values'
''

'how much biomass is bad in the area'
'VERY low amount of bioass = 0-10 MG/ha'
'medium amount of biomass = 10-50 MG/ha'
'medium to high amount of biomass = 50-150 MG/ha'
'high amount of biomass = 150+ MG/ha  forest area'

'LOWER MEANS BAD 0-50'
'80-100 PERCENT CONFIDENCE IS GOOD'


'high amount of biomass = '
print("\nBiomass Value:", biomass_result['MU'])
print("Confidence Value:", confidence_result['PE'])


'{firename,lat,lon}  [evt,evc,[biomass,confidence]]'


'include building material for each fire, for the model incorporate the building material uncertanty to account for brickk and other non flammable materials'
'using microsoft building footprint data'


ee.features.FeatureCollection('Microsoft/USBuildingFootprints/CONUS')












EVC Bands: ['EVC']
EVT Bands: ['EVT']

EVC Value (Cover %): 103
EVT Value (Type Code): 3921

Biomass Value: 1.9107507467269897
Confidence Value: 36


'{firename,lat,lon}  [evt,evc,[biomass,confidence]]'

In [ ]:
''